In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

In [2]:
import re
import string
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [3]:
df = pd.read_csv('./deceptive-opinion.csv')

In [4]:
df.head()

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [5]:
df = df.drop(["hotel", "polarity","source"], axis=1)

In [6]:
df = df.sample(frac=1)

In [7]:
df.head()

,deceptive,text
57,truthful,I just left the Conrad Chicago and have nothin...
929,truthful,Two main issues: (1) understaffed from the str...
728,deceptive,"If you have the money, Amalfi Hotel is the pla..."
885,truthful,We stayed at the Swissotel for a three day wee...
665,deceptive,Our stay at the James in late last August exce...


In [8]:
from sklearn import preprocessing 

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'species'. 
df['deceptive']= label_encoder.fit_transform(df['deceptive']) 

df['deceptive'].unique() 

array([1, 0])

In [9]:
df.head()

,deceptive,text
57,1,I just left the Conrad Chicago and have nothin...
929,1,Two main issues: (1) understaffed from the str...
728,0,"If you have the money, Amalfi Hotel is the pla..."
885,1,We stayed at the Swissotel for a three day wee...
665,0,Our stay at the James in late last August exce...


In [10]:
#dataset description #truthful=1 deceptive=0
df.groupby('deceptive').describe()

text                                                               
          count unique                                                top freq
deceptive                                                                     
0           800    800  I recently stayed at the Hyatt Regency Chicago...    1
1           800    796  My daughter and I woke in the morning wanting ...    2

In [11]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [12]:
# Some preprocesssing that will be common to all the text classification methods

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_char(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

In [13]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [14]:
df['text'] = df['text'].map(lambda a: clean_numbers(a))

In [15]:
df['text'] = df['text'].map(lambda a: clean_char(a))

In [16]:
df['text'] = df['text'].map(lambda a: clean_text(a))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/m1key/nltk_data'
    - '/home/m1key/anaconda3/nltk_data'
    - '/home/m1key/anaconda3/share/nltk_data'
    - '/home/m1key/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [17]:
df['text']

57      I just left the Conrad Chicago and have nothin...
929     Two main issues :   ( 1 )  understaffed from t...
728     If you have the money ,  Amalfi Hotel is the p...
885     We stayed at the Swissotel for a three day wee...
665     Our stay at the James in late last August exce...
                              ...                        
803     My daughter and I woke in the morning wanting ...
600     My husband and I decided to plan a romantic ge...
1239    On a recent business trip to Chicago ,  I had ...
1407    I will never use Ambassador East Hotel again ....
1569    I recently stayed at the Hotel Allegro Chicago...
Name: text, Length: 1600, dtype: object

In [18]:
df.describe()

,deceptive
count,1600.000000
mean,0.500000
std,0.500156
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 57 to 1569
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   deceptive  1600 non-null   int64 
 1   text       1600 non-null   object
dtypes: int64(1), object(1)
memory usage: 37.5+ KB


In [20]:
x = df['text']
y = df['deceptive']

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['deceptive'], random_state=5)
print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 1600
Number of rows in the training set: 1200
Number of rows in the test set: 400


In [24]:
X_train, X_test, y_train, y_test

(814     Our trip was our annual anniversary trip .  Th...
 1285    The Sheraton Chicago Hotel and Towers is not t...
 1277    I had such a horrible experience when I stayed...
 585     For a short time during vacation I had rented ...
 107     We really enjoyed our brief stay at the Millen...
                               ...                        
 1154    The location was good .  Check - in was ok .  ...
 455     I have just returned from my first stay at the...
 1582    Our visit started off on the wrong foot when w...
 1565    I was not pleased with my recent stay at the P...
 1365    For a hotel that touts its luxury location in ...
 Name: text, Length: 1200, dtype: object,
 182     my husband and l stayed here after a very hect...
 1104    October 3rd ,   #  #  #  #  My wife and I stay...
 311     We stay at Hilton for 4 nights last march .  I...
 351     Booked via Priceline . com for a weekend getaw...
 40      Great location and quality place to stay .  Lo...
              

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
print(count_vector)

CountVectorizer()


In [26]:
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

In [27]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB()

In [28]:
predictions = naive_bayes.predict(testing_data)

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
mnbaccuracy = accuracy_score(y_test, predictions)
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.905
Precision score:  0.9055555555555556
Recall score:  0.8858695652173914
F1 score:  0.8956043956043955


In [30]:
from sklearn.svm import SVC 
svc = SVC()
svc.fit(training_data, y_train)

SVC()

In [31]:
svc_predictions = svc.predict(testing_data)

In [32]:
from sklearn.metrics import accuracy_score
svcaccuracy = accuracy_score(y_test,svc_predictions)
print('Accuracy score: ', format(accuracy_score(y_test,svc_predictions)))
print('Precision score: ', format(precision_score(y_test,svc_predictions)))
print('Recall score: ', format(recall_score(y_test, svc_predictions)))
print('F1 score: ', format(f1_score(y_test, svc_predictions)))

Accuracy score:  0.8225
Precision score:  0.7729468599033816
Recall score:  0.8695652173913043
F1 score:  0.8184143222506394


In [33]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(training_data, y_train)

KNeighborsClassifier(n_neighbors=7)

In [34]:
knn_predictions = knn.predict(testing_data)

In [35]:
from sklearn.metrics import accuracy_score
knnaccuracy = accuracy_score(knn_predictions,y_test )
print('Accuracy score: ', format(accuracy_score(knn_predictions,y_test )))
print('Precision score: ', format(precision_score(y_test,knn_predictions)))
print('Recall score: ', format(recall_score(y_test, knn_predictions)))
print('F1 score: ', format(f1_score(y_test, knn_predictions)))

Accuracy score:  0.6825
Precision score:  0.7522123893805309
Recall score:  0.46195652173913043
F1 score:  0.5723905723905723


In [36]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB()
training_data1 = training_data.toarray()
gnb.fit(training_data1, y_train)

GaussianNB()

In [37]:
testing_data1= testing_data.toarray()
gnb_predictions = gnb.predict(testing_data1)

In [38]:
from sklearn.metrics import accuracy_score
gnb_accuracy = accuracy_score(gnb_predictions,y_test )
print('Accuracy score: ', format(accuracy_score(gnb_predictions,y_test )))
print('Precision score: ', format(precision_score(y_test,gnb_predictions)))
print('Recall score: ', format(recall_score(y_test, gnb_predictions)))
print('F1 score: ', format(f1_score(y_test, gnb_predictions)))

Accuracy score:  0.6925
Precision score:  0.6431924882629108
Recall score:  0.7445652173913043
F1 score:  0.690176322418136


In [39]:
# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth = 2)
dtree_model.fit(training_data1, y_train)

DecisionTreeClassifier(max_depth=2)

In [40]:
dtree_predictions = dtree_model.predict(testing_data) 

In [41]:
from sklearn.metrics import accuracy_score
dtree_accuracy = accuracy_score(dtree_predictions,y_test )
print('Accuracy score: ', format(accuracy_score(dtree_predictions,y_test )))
print('Precision score: ', format(precision_score(y_test,dtree_predictions)))
print('Recall score: ', format(recall_score(y_test, dtree_predictions)))
print('F1 score: ', format(f1_score(y_test, dtree_predictions)))

Accuracy score:  0.66
Precision score:  0.6578947368421053
Recall score:  0.5434782608695652
F1 score:  0.5952380952380952


In [42]:
from sklearn.linear_model import SGDClassifier

In [43]:
sgd_clf = SGDClassifier()
sgd_clf.fit(training_data, y_train)

SGDClassifier()

In [44]:
sgdpredicted = sgd_clf.predict(testing_data)

In [45]:
from sklearn.metrics import accuracy_score
sgd_accuracy = accuracy_score(sgdpredicted,y_test )
print('Accuracy score: ', format(accuracy_score(sgdpredicted,y_test )))
print('Precision score: ', format(precision_score(y_test,sgdpredicted)))
print('Recall score: ', format(recall_score(y_test, sgdpredicted)))
print('F1 score: ', format(f1_score(y_test, sgdpredicted)))

Accuracy score:  0.85
Precision score:  0.8229166666666666
Recall score:  0.8586956521739131
F1 score:  0.8404255319148937


In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
lr = LogisticRegression()
lr.fit(training_data, y_train)

/home/m1key/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [48]:
lrpredicted = lr.predict(testing_data)

In [49]:
from sklearn.metrics import accuracy_score
lr_accuracy = accuracy_score(lrpredicted,y_test )
print('Accuracy score: ', format(accuracy_score(lrpredicted,y_test )))
print('Precision score: ', format(precision_score(y_test,lrpredicted)))
print('Recall score: ', format(recall_score(y_test, lrpredicted)))
print('F1 score: ', format(f1_score(y_test, lrpredicted)))

Accuracy score:  0.865
Precision score:  0.8421052631578947
Recall score:  0.8695652173913043
F1 score:  0.8556149732620322


In [50]:
print('Multinomial Naive Bayes:',mnbaccuracy)
print('Gausian Naive Bayes:',gnb_accuracy)
print('Decision tree:',dtree_accuracy)
print('Support Vector Classifier:',svcaccuracy)
print('K-Nearest Neighbour:',knnaccuracy)
print('Stochastic Gradient Descent:',sgd_accuracy)
print('LogisticRegression:',lr_accuracy)

Multinomial Naive Bayes: 0.905
Gausian Naive Bayes: 0.6925
Decision tree: 0.66
Support Vector Classifier: 0.8225
K-Nearest Neighbour: 0.6825
Stochastic Gradient Descent: 0.85
LogisticRegression: 0.865


In [51]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
kmeans.fit(training_data)
y_kmeans = kmeans.predict(training_data)

In [52]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() 

In [53]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [54]:
ps = PorterStemmer()

In [56]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/m1key/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [57]:
stemmed_dataset = []
for i in range(0,1600):
    stemmed_array = df['text'][i].split()
    stemmed = [ps.stem(word) for word in stemmed_array if not word in set(stopwords.words('english'))]
    stemmed = ' '.join(stemmed)
    stemmed_dataset.append(stemmed)

In [58]:
print(stemmed[0:1600])

as former chicagoan , i ' appal amalfi hotel chicago . first , i expect luxuri hospit , neither i receiv . there ' experi design suppos like ' person concierg , ' experi ed terribl . i felt like tri pressur stay day i want . not , i ' understand say time talk fast . when i final got room , i disappoint qualiti furnitur room ' cleanli . i ask maid come give clean towel towel bathroom damp . on top , bed messili done ; i could done better job bed home . i angri point , i pay lot money everi night i stay amalfi , i ' expect greet wet towel . i need use wi - fi download import document , internet surprisingli slow . even basic hotel motel could offer better , mayb even faster internet access . when i final check amalfi , i made sure suppos person concierg knew problem i ' room hotel . i glad see amalfi get smaller mirror i drove away !


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()
X = cv.fit_transform(stemmed_dataset)

In [ ]:
from sklearn.cluster import KMeans
wcss =[]

In [ ]:
for i in range(1,1600):
    kmeans =  KMeans(n_clusters=i, init='k-means++', max_iter = 300, n_init=10, random_state = 0, verbose=True)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)